# Grupowanie państw na podstawie wyglądu ich flag
## Rafał Gulewski, Marek Nowakowski
### Projekt zespołowy - podstawy reprezentacji i analizy danych

Wczytanie niezbędnych pakietów.

In [1]:
import numpy as np
import pandas as pd

1.	Przedstawienie informacji o zbiorze danych (opis atrybutów)

In [2]:
flagDataFile  = open("flag.data", "r") 
flagData = flagDataFile.readlines()
df = pd.DataFrame(columns=['Name', 'Landmass', 'Zone','Area','Population','Language','Religion','Bars','Stripes','Colours','Red','Green','Blue','Gold','White','Black','Orange','Mainhue','Circles','Crosses','Saltires','Quarters','Sunstars','Crescent','Triangle','Icon','Animate','Text','Topleft','Bortight'])
for line in flagData:
    lineData = line.split(',')
    lineData[29] = lineData[29][:-1:]    # Usunięcie znaku \n z końcówki linii
    df = df.append({'Name': lineData[0], 'Landmass': lineData[1], 'Zone': lineData[2],'Area': lineData[3],'Population': lineData[4],'Language': lineData[5],'Religion': lineData[6],'Bars': lineData[7],'Stripes': lineData[8],'Colours': lineData[9],'Red': lineData[10],'Green': lineData[11],'Blue': lineData[12],'Gold': lineData[13],'White': lineData[14],'Black': lineData[15],'Orange': lineData[16],'Mainhue': lineData[17],'Circles': lineData[18],'Crosses': lineData[19],'Saltires': lineData[20],'Quarters': lineData[21],'Sunstars': lineData[22],'Crescent': lineData[23],'Triangle': lineData[24],'Icon': lineData[25],'Animate': lineData[26],'Text': lineData[27],'Topleft': lineData[28],'Bortight': lineData[29]}, ignore_index=True)
flagDataFile.close()
df 

,Name,Landmass,Zone,Area,Population,Language,Religion,Bars,Stripes,Colours,...,Saltires,Quarters,Sunstars,Crescent,Triangle,Icon,Animate,Text,Topleft,Bortight
0,Afghanistan,5,1,648,16,10,2,0,3,5,...,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,...,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,...,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,...,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,...,0,0,0,0,0,0,0,0,blue,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Western-Samoa,6,3,3,0,1,1,0,0,3,...,0,1,5,0,0,0,0,0,blue,red
190,Yugoslavia,3,1,256,22,6,6,0,3,4,...,0,0,1,0,0,0,0,0,blue,red
191,Zaire,4,2,905,28,10,5,0,0,4,...,0,0,0,0,0,1,1,0,green,green
192,Zambia,4,2,753,6,10,5,3,0,4,...,0,0,0,0,0,0,1,0,green,brown


2.	Analiza zbioru ze względu na występowanie danych atrybutów. (ilość wystąpień, średnia itd..)

3.	Przedstawienie zależności między atrybutami

4.	Klasyfikacja i dobór atrybutów znaczących.

5.	Porównanie sposobów grupowania i uzyskanych wyników.

6.	Ocena wyników i sformułowanie wniosków.

7.	Wizualne przedstawienie wyników (grupy z ikonami flag danych państw).